In [1]:
"""
Echo Runner — an original Pygame arcade/puzzle hybrid
Mechanic: Every N seconds, an "echo" appears that replays your own recent path.
Avoid walls, hazards, AND your own echoes while collecting shards.

Controls:
  Move: Arrow keys or WASD
  Pause: P
  Mute/Unmute: M
  Restart (after game over): R
  From menu: Click "Start" or press SPACE/ENTER

Requires: pygame 2.x  (pip install pygame)
"""

import math
import random
import sys
from collections import deque

import pygame

# -------------------- Config --------------------
WIDTH, HEIGHT = 960, 600
FPS = 60
PLAYER_SPEED = 4.2
PLAYER_RADIUS = 12
ECHO_SPAWN_EVERY = 10.0   # seconds between new echoes
ECHO_DELAY = 2.0          # seconds each echo starts behind your current time
ECHO_THICKNESS = 10
ORB_RADIUS = 8
ORB_LIFETIME = 12.0       # seconds before orb relocates
HAZARD_COUNT_START = 3
HAZARD_GROW_EVERY = 4     # add a hazard every N orbs
HAZARD_MIN_SPEED, HAZARD_MAX_SPEED = 1.6, 3.2
PADDING = 40

BG_COLOR = (18, 22, 30)
ARENA_COLOR = (32, 38, 50)
TEXT_COLOR = (230, 236, 250)
ACCENT = (120, 200, 255)
DANGER = (250, 100, 120)
SAFE = (140, 255, 180)
GOLD = (255, 215, 120)

# -------------------- Helpers --------------------
def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def vec_len(x, y):
    return math.hypot(x, y)

def circle_rect_collision(cx, cy, r, rect):
    # Find closest point on rect to circle center
    closest_x = clamp(cx, rect.left, rect.right)
    closest_y = clamp(cy, rect.top, rect.bottom)
    dx, dy = cx - closest_x, cy - closest_y
    return dx * dx + dy * dy <= r * r

def circle_circle_collision(x1, y1, r1, x2, y2, r2):
    return (x1 - x2) ** 2 + (y1 - y2) ** 2 <= (r1 + r2) ** 2

# -------------------- Entities --------------------
class Hazard:
    """Bouncing rectangle hazard."""
    def __init__(self, w, h):
        self.rect = pygame.Rect(0, 0, w, h)
        self.rect.center = (
            random.randint(PADDING, WIDTH - PADDING),
            random.randint(PADDING, HEIGHT - PADDING),
        )
        ang = random.random() * math.tau
        speed = random.uniform(HAZARD_MIN_SPEED, HAZARD_MAX_SPEED)
        self.vx = math.cos(ang) * speed
        self.vy = math.sin(ang) * speed
        self.color = (ARENA_COLOR[0] + 10, ARENA_COLOR[1] + 10, ARENA_COLOR[2] + 10)

    def update(self, bounds):
        self.rect.x += self.vx
        self.rect.y += self.vy
        if self.rect.left < PADDING or self.rect.right > bounds[0] - PADDING:
            self.vx *= -1
        if self.rect.top < PADDING or self.rect.bottom > bounds[1] - PADDING:
            self.vy *= -1

    def draw(self, surf):
        pygame.draw.rect(surf, self.color, self.rect, border_radius=10)
        pygame.draw.rect(surf, DANGER, self.rect, 2, border_radius=10)

class Orb:
    """Collectible that relocates periodically."""
    def __init__(self):
        self.pos = self.random_pos()
        self.timer = 0.0

    def random_pos(self):
        return [
            random.randint(PADDING + 30, WIDTH - PADDING - 30),
            random.randint(PADDING + 30, HEIGHT - PADDING - 30),
        ]

    def update(self, dt):
        self.timer += dt
        if self.timer >= ORB_LIFETIME:
            self.pos = self.random_pos()
            self.timer = 0.0

    def draw(self, surf):
        pygame.draw.circle(surf, GOLD, self.pos, ORB_RADIUS)
        # gentle pulse ring
        t = pygame.time.get_ticks() / 1000.0
        r = ORB_RADIUS + 3 + int(2 * math.sin(t * 2))
        pygame.draw.circle(surf, (255, 255, 255), self.pos, r, 1)

class Echo:
    """Replays a recorded path with a time offset. Deadly to the player."""
    def __init__(self, path_buffer, start_offset_seconds, color):
        self.path_buffer = path_buffer  # deque[(x, y)]
        self.offset_frames = int(start_offset_seconds * FPS)
        self.frame = 0
        self.color = color
        self.alive = True

    def get_pos(self):
        idx = self.frame - self.offset_frames
        if idx < 0 or idx >= len(self.path_buffer):
            return None
        return self.path_buffer[idx]

    def update(self):
        self.frame += 1

    def draw(self, surf):
        pos = self.get_pos()
        if pos:
            x, y = pos
            pygame.draw.circle(surf, self.color, (int(x), int(y)), PLAYER_RADIUS - 2)
            # trail
            for k in range(6, 24, 6):
                idx = self.frame - self.offset_frames - k
                if 0 <= idx < len(self.path_buffer):
                    tx, ty = self.path_buffer[idx]
                    pygame.draw.circle(surf, self.color, (int(tx), int(ty)), 6, 1)

# -------------------- Game --------------------
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Echo Runner — avoid your past")
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont("consolas", 22)
        self.big = pygame.font.SysFont("consolas", 48, bold=True)
        self.medium = pygame.font.SysFont("consolas", 28)

        # sounds (generated simple tones)
        self.muted = False
        self.snd_pick, self.snd_hit, self.snd_spawn = self._make_sounds()

        self.reset()

        # UI
        self.state = "MENU"
        self.start_button = pygame.Rect(0, 0, 220, 64)
        self.start_button.center = (WIDTH // 2, HEIGHT // 2 + 60)

    def _make_sounds(self):
        # Simple procedural beeps to avoid assets
        try:
            pygame.mixer.init()
            def tone(freq, ms, vol=0.25):
                # create mono waveform
                sample_rate = 22050
                n_samples = int(sample_rate * ms / 1000)
                buf = bytearray()
                for i in range(n_samples):
                    v = int(127 * math.sin(2 * math.pi * freq * (i / sample_rate)))
                    buf += int(v * vol + 128).to_bytes(1, "little", signed=False)
                snd = pygame.mixer.Sound(buffer=bytes(buf))
                return snd
            return tone(880, 90), tone(120, 200, 0.35), tone(520, 140)
        except Exception:
            # Fallback: no sound
            class Silent:
                def play(self): pass
            s = Silent()
            return s, s, s

    def reset(self):
        self.player_x = WIDTH // 2
        self.player_y = HEIGHT // 2
        self.vx = 0.0
        self.vy = 0.0
        self.score = 0
        self.time_alive = 0.0
        self.orb = Orb()
        self.hazards = [Hazard(random.randint(40, 90), random.randint(26, 70))
                        for _ in range(HAZARD_COUNT_START)]
        self.path_buffer = deque(maxlen=int((ECHO_DELAY + 60) * FPS))  # keep 1 min of history + delay
        self.echoes = []
        self.echo_timer = 0.0
        self.paused = False

    def spawn_echo(self):
        # Each echo uses a different pastel color
        hue = random.random()
        color = self._hsv_to_rgb(hue, 0.35, 1.0)
        self.echoes.append(Echo(self.path_buffer, ECHO_DELAY, color))
        if not self.muted:
            self.snd_spawn.play()

    @staticmethod
    def _hsv_to_rgb(h, s, v):
        i = int(h * 6)
        f = h * 6 - i
        p = int(255 * v * (1 - s))
        q = int(255 * v * (1 - f * s))
        t = int(255 * v * (1 - (1 - f) * s))
        v = int(255 * v)
        i %= 6
        if i == 0: return (v, t, p)
        if i == 1: return (q, v, p)
        if i == 2: return (p, v, t)
        if i == 3: return (p, q, v)
        if i == 4: return (t, p, v)
        return (v, p, q)

    # -------------- State handlers --------------
    def run(self):
        while True:
            dt = self.clock.tick(FPS) / 1000.0
            if self.state == "MENU":
                self.update_menu(dt)
                self.draw_menu()
            elif self.state == "RUN":
                self.update_run(dt)
                self.draw_run()
            elif self.state == "PAUSE":
                self.update_pause(dt)
                self.draw_pause()
            elif self.state == "GAMEOVER":
                self.update_gameover(dt)
                self.draw_gameover()
            pygame.display.flip()

    def update_menu(self, dt):
        for ev in pygame.event.get():
            if ev.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if ev.type == pygame.KEYDOWN:
                if ev.key in (pygame.K_SPACE, pygame.K_RETURN):
                    self.reset()
                    self.state = "RUN"
            if ev.type == pygame.MOUSEBUTTONDOWN and ev.button == 1:
                if self.start_button.collidepoint(ev.pos):
                    self.reset()
                    self.state = "RUN"

    def draw_menu(self):
        self.screen.fill(BG_COLOR)
        self._draw_arena()
        title = self.big.render("E C H O   R U N N E R", True, TEXT_COLOR)
        self.screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 120))
        sub = self.medium.render("Avoid hazards… and your own past.", True, ACCENT)
        self.screen.blit(sub, (WIDTH // 2 - sub.get_width() // 2, 180))

        # Instructions
        lines = [
            "Move: WASD / Arrow Keys",
            "Pause: P    Mute: M",
            "Collect shards to score. Every 10s an echo spawns",
            "that replays your recent path. Touching any echo = GAME OVER.",
        ]
        for i, t in enumerate(lines):
            surf = self.font.render(t, True, TEXT_COLOR)
            self.screen.blit(surf, (WIDTH // 2 - surf.get_width() // 2, 240 + i * 26))

        # Start button
        pygame.draw.rect(self.screen, ACCENT, self.start_button, border_radius=16)
        label = self.medium.render("START", True, (10, 20, 30))
        self.screen.blit(label, (
            self.start_button.centerx - label.get_width() // 2,
            self.start_button.centery - label.get_height() // 2
        ))

        hint = self.font.render("Tip: plan future you around past you.", True, SAFE)
        self.screen.blit(hint, (WIDTH // 2 - hint.get_width() // 2, HEIGHT - 80))

        credit = self.font.render("Original mechanic & code — generated for you", True, (160, 170, 190))
        self.screen.blit(credit, (WIDTH // 2 - credit.get_width() // 2, HEIGHT - 40))

    def update_run(self, dt):
        for ev in pygame.event.get():
            if ev.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if ev.type == pygame.KEYDOWN:
                if ev.key == pygame.K_p:
                    self.state = "PAUSE"
                if ev.key == pygame.K_m:
                    self.muted = not self.muted

        # Movement
        keys = pygame.key.get_pressed()
        dx = (keys[pygame.K_RIGHT] or keys[pygame.K_d]) - (keys[pygame.K_LEFT] or keys[pygame.K_a])
        dy = (keys[pygame.K_DOWN] or keys[pygame.K_s]) - (keys[pygame.K_UP] or keys[pygame.K_w])
        if dx or dy:
            length = math.hypot(dx, dy)
            self.vx = (dx / (length or 1)) * PLAYER_SPEED
            self.vy = (dy / (length or 1)) * PLAYER_SPEED
        else:
            self.vx *= 0.85
            self.vy *= 0.85

        self.player_x = clamp(self.player_x + self.vx, PADDING + PLAYER_RADIUS, WIDTH - PADDING - PLAYER_RADIUS)
        self.player_y = clamp(self.player_y + self.vy, PADDING + PLAYER_RADIUS, HEIGHT - PADDING - PLAYER_RADIUS)

        # Update timers & record path
        self.time_alive += dt
        self.echo_timer += dt
        self.path_buffer.append((self.player_x, self.player_y))

        # Spawn echoes periodically (only after enough history gathered)
        if self.echo_timer >= ECHO_SPAWN_EVERY and len(self.path_buffer) > int(ECHO_DELAY * FPS) + 10:
            self.echo_timer = 0.0
            self.spawn_echo()

        # Update entities
        self.orb.update(dt)
        for hz in self.hazards:
            hz.update((WIDTH, HEIGHT))
        for ec in self.echoes:
            ec.update()

        # Difficulty scaling: add hazards per few orbs
        if self.score // HAZARD_GROW_EVERY + HAZARD_COUNT_START > len(self.hazards):
            self.hazards.append(Hazard(random.randint(40, 90), random.randint(26, 70)))

        # Collisions: player with orb
        if circle_circle_collision(self.player_x, self.player_y, PLAYER_RADIUS, self.orb.pos[0], self.orb.pos[1], ORB_RADIUS):
            self.score += 1
            self.orb.pos = self.orb.random_pos()
            self.orb.timer = 0.0
            if not self.muted:
                self.snd_pick.play()

        # Collisions: player with hazards
        for hz in self.hazards:
            if circle_rect_collision(self.player_x, self.player_y, PLAYER_RADIUS, hz.rect):


SyntaxError: incomplete input (640545326.py, line 348)

In [5]:
"""
Echo Runner — an original Pygame arcade/puzzle hybrid
Mechanic: Every N seconds, an "echo" appears that replays your own recent path.
Avoid walls, hazards, AND your own echoes while collecting shards.

Controls:
  Move: Arrow keys or WASD
  Pause: P
  Mute/Unmute: M
  Restart (after game over): R
  From menu: Click "Start" or press SPACE/ENTER

Requires: pygame 2.x  (pip install pygame)
"""

import math
import random
import sys
from collections import deque

import pygame

# -------------------- Config --------------------
WIDTH, HEIGHT = 960, 600
FPS = 60
PLAYER_SPEED = 4.2
PLAYER_RADIUS = 12
ECHO_SPAWN_EVERY = 10.0   # seconds between new echoes
ECHO_DELAY = 2.0          # seconds each echo starts behind your current time
ORB_RADIUS = 8
ORB_LIFETIME = 12.0       # seconds before orb relocates
HAZARD_COUNT_START = 3
HAZARD_GROW_EVERY = 4     # add a hazard every N orbs
HAZARD_MIN_SPEED, HAZARD_MAX_SPEED = 1.6, 3.2
PADDING = 40

BG_COLOR = (18, 22, 30)
ARENA_COLOR = (32, 38, 50)
TEXT_COLOR = (230, 236, 250)
ACCENT = (120, 200, 255)
DANGER = (250, 100, 120)
SAFE = (140, 255, 180)
GOLD = (255, 215, 120)

# -------------------- Helpers --------------------
def clamp(v, lo, hi):
    return max(lo, min(hi, v))

def circle_rect_collision(cx, cy, r, rect):
    closest_x = clamp(cx, rect.left, rect.right)
    closest_y = clamp(cy, rect.top, rect.bottom)
    dx, dy = cx - closest_x, cy - closest_y
    return dx * dx + dy * dy <= r * r

def circle_circle_collision(x1, y1, r1, x2, y2, r2):
    return (x1 - x2) ** 2 + (y1 - y2) ** 2 <= (r1 + r2) ** 2

# -------------------- Entities --------------------
class Hazard:
    """Bouncing rectangle hazard."""
    def __init__(self, w, h):
        self.rect = pygame.Rect(0, 0, w, h)
        self.rect.center = (
            random.randint(PADDING, WIDTH - PADDING),
            random.randint(PADDING, HEIGHT - PADDING),
        )
        ang = random.random() * math.tau
        speed = random.uniform(HAZARD_MIN_SPEED, HAZARD_MAX_SPEED)
        self.vx = math.cos(ang) * speed
        self.vy = math.sin(ang) * speed
        self.color = (ARENA_COLOR[0] + 10, ARENA_COLOR[1] + 10, ARENA_COLOR[2] + 10)

    def update(self, bounds):
        self.rect.x += self.vx
        self.rect.y += self.vy
        if self.rect.left < PADDING or self.rect.right > bounds[0] - PADDING:
            self.vx *= -1
        if self.rect.top < PADDING or self.rect.bottom > bounds[1] - PADDING:
            self.vy *= -1

    def draw(self, surf):
        pygame.draw.rect(surf, self.color, self.rect, border_radius=10)
        pygame.draw.rect(surf, DANGER, self.rect, 2, border_radius=10)

class Orb:
    """Collectible that relocates periodically."""
    def __init__(self):
        self.pos = self.random_pos()
        self.timer = 0.0

    def random_pos(self):
        return [
            random.randint(PADDING + 30, WIDTH - PADDING - 30),
            random.randint(PADDING + 30, HEIGHT - PADDING - 30),
        ]

    def update(self, dt):
        self.timer += dt
        if self.timer >= ORB_LIFETIME:
            self.pos = self.random_pos()
            self.timer = 0.0

    def draw(self, surf):
        pygame.draw.circle(surf, GOLD, self.pos, ORB_RADIUS)
        # gentle pulse ring
        t = pygame.time.get_ticks() / 1000.0
        r = ORB_RADIUS + 3 + int(2 * math.sin(t * 2))
        pygame.draw.circle(surf, (255, 255, 255), self.pos, r, 1)

class Echo:
    """Replays a recorded path with a time offset. Deadly to the player."""
    def __init__(self, path_buffer, start_offset_seconds, color):
        self.path_buffer = path_buffer
        self.offset_frames = int(start_offset_seconds * FPS)
        self.frame = 0
        self.color = color

    def get_pos(self):
        idx = self.frame - self.offset_frames
        if idx < 0 or idx >= len(self.path_buffer):
            return None
        return self.path_buffer[idx]

    def update(self):
        self.frame += 1

    def draw(self, surf):
        pos = self.get_pos()
        if pos:
            x, y = pos
            pygame.draw.circle(surf, self.color, (int(x), int(y)), PLAYER_RADIUS - 2)

# -------------------- Game --------------------
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Echo Runner — avoid your past")
        self.screen = pygame.display.set_mode((WIDTH, HEIGHT))
        self.clock = pygame.time.Clock()
        self.font = pygame.font.SysFont("consolas", 22)
        self.big = pygame.font.SysFont("consolas", 48, bold=True)
        self.medium = pygame.font.SysFont("consolas", 28)

        self.reset()
        self.state = "MENU"

    def reset(self):
        self.player_x = WIDTH // 2
        self.player_y = HEIGHT // 2
        self.vx, self.vy = 0.0, 0.0
        self.score = 0
        self.time_alive = 0.0
        self.orb = Orb()
        self.hazards = [Hazard(random.randint(40, 90), random.randint(26, 70))
                        for _ in range(HAZARD_COUNT_START)]
        self.path_buffer = deque(maxlen=int((ECHO_DELAY + 60) * FPS))
        self.echoes = []
        self.echo_timer = 0.0

    def spawn_echo(self):
        hue = random.random()
        color = (int(200 + 55 * hue), int(150 + 100 * hue), 255)
        self.echoes.append(Echo(self.path_buffer, ECHO_DELAY, color))

    def run(self):
        while True:
            dt = self.clock.tick(FPS) / 1000.0
            if self.state == "MENU":
                self.update_menu()
                self.draw_menu()
            elif self.state == "RUN":
                self.update_run(dt)
                self.draw_run()
            elif self.state == "GAMEOVER":
                self.update_gameover()
                self.draw_gameover()
            pygame.display.flip()

    # -------- MENU --------
    def update_menu(self):
        for ev in pygame.event.get():
            if ev.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if ev.type == pygame.KEYDOWN and ev.key in (pygame.K_SPACE, pygame.K_RETURN):
                self.reset()
                self.state = "RUN"

    def draw_menu(self):
        self.screen.fill(BG_COLOR)
        t = self.big.render("ECHO RUNNER", True, TEXT_COLOR)
        self.screen.blit(t, (WIDTH//2 - t.get_width()//2, HEIGHT//2 - 50))
        s = self.font.render("Press SPACE to start", True, ACCENT)
        self.screen.blit(s, (WIDTH//2 - s.get_width()//2, HEIGHT//2 + 10))

    # -------- RUN --------
    def update_run(self, dt):
        for ev in pygame.event.get():
            if ev.type == pygame.QUIT:
                pygame.quit(); sys.exit()

        keys = pygame.key.get_pressed()
        dx = (keys[pygame.K_RIGHT] or keys[pygame.K_d]) - (keys[pygame.K_LEFT] or keys[pygame.K_a])
        dy = (keys[pygame.K_DOWN] or keys[pygame.K_s]) - (keys[pygame.K_UP] or keys[pygame.K_w])
        if dx or dy:
            length = math.hypot(dx, dy)
            self.vx = (dx / (length or 1)) * PLAYER_SPEED
            self.vy = (dy / (length or 1)) * PLAYER_SPEED

        self.player_x = clamp(self.player_x + self.vx, PADDING + PLAYER_RADIUS, WIDTH - PADDING - PLAYER_RADIUS)
        self.player_y = clamp(self.player_y + self.vy, PADDING + PLAYER_RADIUS, HEIGHT - PADDING - PLAYER_RADIUS)

        # timers
        self.time_alive += dt
        self.echo_timer += dt
        self.path_buffer.append((self.player_x, self.player_y))

        # spawn echoes
        if self.echo_timer >= ECHO_SPAWN_EVERY and len(self.path_buffer) > int(ECHO_DELAY * FPS) + 10:
            self.echo_timer = 0.0
            self.spawn_echo()

        # update
        self.orb.update(dt)
        for hz in self.hazards: hz.update((WIDTH, HEIGHT))
        for ec in self.echoes: ec.update()

        # collisions: orb
        if circle_circle_collision(self.player_x, self.player_y, PLAYER_RADIUS, self.orb.pos[0], self.orb.pos[1], ORB_RADIUS):
            self.score += 1
            self.orb.pos = self.orb.random_pos()
            self.orb.timer = 0.0

        # collisions: hazards
        for hz in self.hazards:
            if circle_rect_collision(self.player_x, self.player_y, PLAYER_RADIUS, hz.rect):
                self.state = "GAMEOVER"
                self.reason = "Hit a hazard!"
                return

        # collisions: echoes
        for ec in self.echoes:
            pos = ec.get_pos()
            if pos and circle_circle_collision(self.player_x, self.player_y, PLAYER_RADIUS, pos[0], pos[1], PLAYER_RADIUS - 1):
                self.state = "GAMEOVER"
                self.reason = "Collided with your echo!"
                return

    def draw_run(self):
        self.screen.fill(ARENA_COLOR)
        self.orb.draw(self.screen)
        for hz in self.hazards: hz.draw(self.screen)
        for ec in self.echoes: ec.draw(self.screen)
        pygame.draw.circle(self.screen, SAFE, (int(self.player_x), int(self.player_y)), PLAYER_RADIUS)
        s = self.font.render(f"Score: {self.score}", True, TEXT_COLOR)
        self.screen.blit(s, (10, 10))

    # -------- GAMEOVER --------
    def update_gameover(self):
        for ev in pygame.event.get():
            if ev.type == pygame.QUIT:
                pygame.quit(); sys.exit()
            if ev.type == pygame.KEYDOWN and ev.key == pygame.K_r:
                self.reset()
                self.state = "RUN"

    def draw_gameover(self):
        self.screen.fill(BG_COLOR)
        t = self.big.render("GAME OVER", True, DANGER)
        self.screen.blit(t, (WIDTH//2 - t.get_width()//2, HEIGHT//2 - 60))
        r = self.font.render(self.reason, True, ACCENT)
        self.screen.blit(r, (WIDTH//2 - r.get_width()//2, HEIGHT//2))
        s = self.font.render("Press R to restart", True, TEXT_COLOR)
        self.screen.blit(s, (WIDTH//2 - s.get_width()//2, HEIGHT//2 + 40))

# -------------------- Main --------------------
if __name__ == "__main__":
    Game().run()


SystemExit: 